In [1]:
import torch
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
prompt_text = input("Input something: ")
encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")

Input something:  Hello this is a test


In [3]:
num = int(input("How deep to search? "))

How deep to search?  10


In [4]:
prediction_scores, past = model.forward(encoded_prompt)
print([tokenizer.decode(index.item()) for index in prediction_scores[0, -1].topk(num).indices])

['[ u n u s e d 1 7 1 ]', '[ u n u s e d 4 9 6 ]', '[ u n u s e d 3 1 7 ]', '[ u n u s e d 2 2 7 ]', '[ u n u s e d 6 2 5 ]', '[ u n u s e d 2 6 3 ]', '[ u n u s e d 3 2 3 ]', '[ u n u s e d 6 6 2 ]', '[ u n u s e d 6 5 2 ]', '[ u n u s e d 1 1 9 ]']


In [5]:
prediction_scores.shape

torch.Size([1, 5, 768])